In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import (
    NMF, 
    LatentDirichletAllocation,
    TruncatedSVD
)
import requests, gensim, spacy, json, nltk
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
# warnings.filterwarnings('ignore')
from sklearn.model_selection import GridSearchCV
from nltk.tokenize import wordpunct_tokenize
from nltk.metrics import ConfusionMatrix
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from nltk.stem import PorterStemmer, WordNetLemmatizer
import matplotlib.pyplot as plt
from functools import reduce
from io import StringIO
from time import time
import pandas as pd
import numpy as np
import re
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import pandas as pd

In [ ]:
# Get stopwords for Porguese and add prominent missing stopword (verb "to be")
# Full list of stopwords printed in appendix
pt_stopwords = nltk.corpus.stopwords.words('portuguese')
en_stopwords = nltk.corpus.stopwords.words('english')
html_elements = ['li', 'ul']
pt_stopwords.append("ser")
nltk_stopwords = pt_stopwords + en_stopwords + html_elements

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
df = pd.read_csv('text-added-news.csv')
docs_raw = df['text'].astype(str)

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                stop_words = nltk_stopwords,
                                lowercase = True,
                                token_pattern = r'\b[a-zA-Z]{3,}\b',
                                max_df = 0.5, 
                                min_df = 10)
dtm_tf = tf_vectorizer.fit_transform(docs_raw)
tfidf_vectorizer = TfidfVectorizer(**tf_vectorizer.get_params())
dtm_tfidf = tfidf_vectorizer.fit_transform(docs_raw)

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/shared-libs/python3.7/py/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:401: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ate', 'eramos', 'estao', 'estavamos', 'estiveramos', 'estivessemos', 'foramos', 'fossemos', 'hao', 'houveramos', 'houverao', 'houveriamos', 'houvessemos', 'nao', 'sao', 'sera', 'serao', 'seriamos', 'tambem', 'tera', 'terao', 'teriamos', 'tinhamos', 'tiveramos', 'tivessemos', 'voce', 'voces'] not in stop_words.
  % sorted(inconsistent)
/shared-libs/python3.7/py/lib/python3.7/site-packages/sklearn/feature_extrac

In [ ]:
# for TF DTM
lda_tf = LatentDirichletAllocation(n_components=10, random_state=0)
lda_tf.fit(dtm_tf)
# for TFIDF DTM
lda_tfidf = LatentDirichletAllocation(n_components=10, random_state=0)
lda_tfidf.fit(dtm_tfidf)

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


LatentDirichletAllocation(random_state=0)

In [ ]:
pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer)

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/shared-libs/python3.7/py/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7     -0.208841  0.010285       1        1  19.390072
5      0.133812  0.037925       2        1  14.753274
6      0.079652  0.143668       3        1  10.727075
2     -0.112879  0.134169       4        1  10.672778
0      0.104272 -0.110206       5        1   9.821837
9     -0.042891 -0.092759       6        1   9.202949
1     -0.096459 -0.140284       7        1   9.201035
8      0.128727 -0.047449       8        1   6.345261
4      0.039929  0.038326       9        1   5.758200
3     -0.025322  0.026327      10        1   4.127520, topic_info=            Term         Freq        Total Category  logprob  loglift
6207     policia  1709.000000  1709.000000  Default  30.0000  30.0000
6404  presidente  2294.000000  2294.000000  Default  29.0000  29.0000
4026     governo  2286.000000  2286.000000  Default  28.0000  28.0000
6123         pib   673.000000   673.000000  Default  27.0000  27.0000
1104   bolsonaro  1435.000000  1435.000000  Default  26.0000  26.0000
...          ...          ...          ...      ...      ...      ...
8265         vez    71.354181   864.202546  Topic10  -5.9289   0.6933
6240      porque    69.664154  1015.871329  Topic10  -5.9528   0.5077
7802       tempo    67.299855   709.158632  Topic10  -5.9874   0.8326
3583       fazer    67.920292   867.499041  Topic10  -5.9782   0.6402
2837     durante    68.472528  1215.710131  Topic10  -5.9701   0.3108

[804 rows x 6 columns], token_table=      Topic      Freq           Term
term                                
10        2  0.933218  abastecimento
10        3  0.049117  abastecimento
10        9  0.016372  abastecimento
40        2  0.049311            abu
40        9  0.147932            abu
...     ...       ...            ...
8378      7  0.033742           voce
8378      8  0.027315           voce
8378      9  0.019281           voce
8378     10  0.244226           voce
8424      1  0.996064      wanderson

[2727 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 6, 7, 3, 1, 10, 2, 9, 5, 4])

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=2392c230-4672-47ef-bbd2-e97d3feaff76' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>